<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/%E2%98%85%EC%B5%9C%EC%A2%85/(%EC%A7%84%EC%A7%9C_%EC%B5%9C%EC%A2%85)MLM_%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
pip install transformers torch

In [36]:
pip install konlpy

In [37]:
from transformers import pipeline, AutoTokenizer
from konlpy.tag import Okt
import random

def mask_and_predict(sentence, model_name="beomi/KcBERT-base", num_masks=2, top_k=5):

    # 형태소 분석기(Okt) 사용하여 단어 분리
    okt = Okt()
    words = okt.morphs(sentence)  # 형태소 단위로 분리

    # BERT 모델 및 토크나이저 로드
    mlm_pipeline = pipeline("fill-mask", model=model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 마스킹할 후보 단어 선택 (명사, 동사 우선)
    maskable_indices = [i for i, word in enumerate(words) if word not in ['을', '를', '이', '가', '은', '는', '에', '도', '의', '한', '가장']]

    # 마스킹할 단어 개수 설정
    num_masks = min(num_masks, len(maskable_indices))
    mask_indices = random.sample(maskable_indices, num_masks)

    # 마스킹 적용
    for idx in mask_indices:
        words[idx] = "[MASK]"

    # 마스킹된 문장 생성
    masked_sentence = " ".join(words)

    # MLM 모델로 예측 수행
    predictions = mlm_pipeline(masked_sentence, top_k=top_k)

    # 여러 개의 [MASK]를 예측하여 문장 복원
    new_sentence = masked_sentence
    for i, mask_preds in enumerate(predictions):
        best_pred = random.choice([pred["token_str"] for pred in mask_preds[:3]])  # Top 3 중 랜덤 선택
        new_sentence = new_sentence.replace("[MASK]", best_pred, 1)

    return new_sentence


# 데이터 불러오기

In [38]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'  'My Drive'   NSMC_0.02.pkl   NSMC_0.5.pkl   이전   졸업논문


In [39]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_0.5_증분대상_2500개.csv', encoding='utf-8-sig')

In [40]:
df['document'].isnull().sum()

0

In [ ]:
df['MLM_aug'] = df['document'].apply(mask_and_predict)

Some weights of the model checkpoint at beomi/KcBERT-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Some weights of the model checkpoint at beomi/KcBERT-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the c

In [ ]:
df.head(5)

In [ ]:
df.to_csv('NSMC_0.5_aug_mlm_kcbert_고도화.csv', encoding='utf-8-sig')